In [1]:
from unet_prob import *

Using TensorFlow backend.


In [2]:
unet_prob = Unet_Prob()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 512, 512, 64) 36928       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

In [3]:
#get data
TRAIN_PATH_IN = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/train/in/'
VAL_PATH_IN = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/val/in/'
TEST_PATH_IN = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/test/in/'

TRAIN_PATH_REF = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/train/ref/'
VAL_PATH_REF = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/val/ref/'
TEST_PATH_REF = '/media/oala/4TB/DATA/experiments-hhi/uncertainty-quant/ellipsoid-toy/denoising/test/ref/'


def get_data(path):
    TRAIN_PATH = path
    train_names = next(os.walk(TRAIN_PATH))[2]
    X_train = np.zeros((len(train_names), 512, 512), dtype=np.float64)
    print('Getting and resizing train images and masks ... ')
    for image_name, count in zip(train_names, range(len(train_names))):
        img = imread(TRAIN_PATH + image_name)
        #img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_train[count] = img
        if count % 500 == 0:
            print('Done with # ', count)
    X_train = X_train[:,:,:,np.newaxis]
    
    return X_train

X_train = get_data(TRAIN_PATH_IN)
Y_train = get_data(TRAIN_PATH_REF)

X_val = get_data(VAL_PATH_IN)
Y_val = get_data(VAL_PATH_REF)

Getting and resizing train images and masks ... 
Done with #  0
Done with #  500
Done with #  1000
Done with #  1500
Getting and resizing train images and masks ... 
Done with #  0
Done with #  500
Done with #  1000
Done with #  1500
Getting and resizing train images and masks ... 
Done with #  0
Getting and resizing train images and masks ... 
Done with #  0


In [4]:
print(X_train.shape)

(1600, 512, 512, 1)


In [5]:
#get mean
#mean_X_train = np.mean(X_train, axis=0)
#mean_Y_train = np.mean(Y_train, axis=0)

#print(mean_X_train.shape, mean_Y_train.shape)

In [6]:
#demean train data
#X_train -= mean_X_train
#Y_train -= mean_Y_train

In [7]:
#get std of demeaned train data
#std_X_train = np.std(X_train, axis = 0)
#std_Y_train = np.std(X_train, axis = 0)

In [8]:
#normalize train with std
#X_train /= std_X_train
#Y_train /= std_Y_train

In [9]:
#demean val, normalize val with std
#X_val = (X_val-mean_X_train)/std_X_train
#Y_val = (Y_val-mean_Y_train)/std_Y_train

In [10]:
#scale 0 to 1
X_train = X_train / 65535.
Y_train = Y_train / 65535.
X_val = X_val / 65535.
Y_val = Y_val / 65535.

In [11]:
print(np.amax(X_train), np.amin(X_train), np.amax(Y_train), np.amin(Y_train))
print(np.amax(X_val), np.amin(X_val), np.amax(Y_val), np.amin(Y_val))

1.0 0.0 1.0 0.0
1.0 0.0 1.0 0.0


In [12]:
unet_prob.train(70, 1, X_train, Y_train, X_val, Y_val, X_val, Y_val)

Epoch 0 of 70
Train loss: -1020945.4982414246
Val loss: -1788330.658125
Took 812.6850214004517 seconds


Epoch 1 of 70
Train loss: -1800789.16984375
Val loss: -1805331.979375
Took 810.7223076820374 seconds


Epoch 2 of 70
Train loss: -1806455.60921875
Val loss: -1807067.381875
Took 810.8156745433807 seconds


Epoch 3 of 70
Train loss: -1807374.8478125
Val loss: -1807521.485
Took 810.9283299446106 seconds


Epoch 4 of 70
Train loss: -1807654.9528125
Val loss: -1807681.4325
Took 810.9790847301483 seconds


Epoch 5 of 70
Train loss: -1807757.40078125
Val loss: -1807742.435625
Took 810.8757779598236 seconds


Epoch 6 of 70
Train loss: -1807797.78078125
Val loss: -1807767.2325
Took 810.9148814678192 seconds


Epoch 7 of 70
Train loss: -1807815.198203125
Val loss: -1807781.419375
Took 810.9028398990631 seconds


Epoch 8 of 70
Train loss: -1807932.173203125
Val loss: -1808243.925
Took 810.9771237373352 seconds


Epoch 9 of 70
Train loss: -1808904.384609375
Val loss: -1809256.084375
Took 808.0

In [16]:
#unet_prob.save('unet_prob_linearprevar-constrainedmean-0to1inAndOut.h5')

In [12]:
unet_prob.unet.load_weights('unet_prob_linearprevar-constrainedmean-0to1inAndOut.h5', by_name=True)

In [13]:
#a = unet_prob.unet.get_weights(conv1)

In [14]:
imgs = unet_prob.unet.predict(X_val, batch_size=1)[:,:,:,0]

In [19]:
imgs = imgs[:,:,:,np.newaxis]
print(imgs.shape)
print(Y_val.shape)

(200, 512, 512, 1)
(200, 512, 512, 1)


In [20]:
def SNR(rec, oracle):
    sumP = np.sum(oracle)
    sumI = np.sum(rec)
    sumIP = np.sum(oracle*rec)
    sumI2 = np.sum(rec**2)
    A = np.array([[sumI2, sumI],[sumI, oracle.size]])
    b = np.array([[sumIP],[sumP]])
    c,_,_,_ = np.linalg.lstsq(A,b)
    #print(c)
    rec = c[0]*rec + c[1]
    err = np.sum((oracle - rec)**2)
    SNR = 10*np.log10(np.sum(oracle**2)/err)

    return SNR

def avg_SNR(REC, ORACLE):
    N = REC.shape[0]
    total_SNR = 0
    for i in range(N):
        total_SNR += SNR(REC[i], ORACLE[i])

    return total_SNR/N

In [21]:
print(avg_SNR(X_val, Y_val))

/home/oala/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


4.721631451381818


In [22]:
print(avg_SNR(imgs, Y_val))

/home/oala/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


18.795853379284726
